# **Importing essential libraries**

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import models, layers, optimizers
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re
%matplotlib inline 
from sklearn.metrics import confusion_matrix
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [2]:
import random
import csv
#importing datasets using commands below
data2 =pd.read_csv('/content/negative(1).csv')
data1=pd.read_csv('/content/positive(1).csv')
Data=data1.append(data2, ignore_index=True) #appending both files to make it as one and then spliting of the data set would be easier 
Data.sample(frac=1) #shuffling all the rows in Data 
Train=Data.iloc[:35000,:] #training set
Test=Data.iloc[:35000,:] #testing set
Data

,Review,Label
0,Better than Wolff's Kasha. I grew up eating Ka...,1
1,It was such good product. Came in two differen...,1
2,MMMM Yes all chocolate is good.<br />But some ...,1
3,"This is, as all of their cereals I've ordered ...",1
4,Whoever Photoshopped the cookie on the front o...,1
...,...,...
49995,"With so many other glowing reviews, I feel a l...",0
49996,Giving it 3 stars because for an 'All Purpose'...,0
49997,"more affordable than other bully sticks, but m...",0
49998,I got this for my daughter because she love bl...,0


In [3]:
Data['Review_1']= Data['Review'].str.replace("[^a-zA-Z#!,.]", " ")  #removing all signs 
Data['Review_1']= Data['Review'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3])) #cleaning text
Data

,Review,Label,Review_1
0,Better than Wolff's Kasha. I grew up eating Ka...,1,Better than Wolff's Kasha. grew eating Kasha e...
1,It was such good product. Came in two differen...,1,such good product. Came different boxes descri...
2,MMMM Yes all chocolate is good.<br />But some ...,1,MMMM chocolate good.<br />But some chocolates ...
3,"This is, as all of their cereals I've ordered ...",1,"This their cereals I've ordered fresh, high qu..."
4,Whoever Photoshopped the cookie on the front o...,1,Whoever Photoshopped cookie front package true...
...,...,...,...
49995,"With so many other glowing reviews, I feel a l...",0,"With many other glowing reviews, feel little a..."
49996,Giving it 3 stars because for an 'All Purpose'...,0,"Giving stars because 'All Purpose' bake mix, r..."
49997,"more affordable than other bully sticks, but m...",0,"more affordable than other bully sticks, offic..."
49998,I got this for my daughter because she love bl...,0,this daughter because love blood chocolate thi...


In [4]:
Token_Data = Data['Review_1'].apply(lambda x: x.split())
Token_Data #tokening data 

0        [Better, than, Wolff's, Kasha., grew, eating, ...
1        [such, good, product., Came, different, boxes,...
2        [MMMM, chocolate, good.<br, />But, some, choco...
3        [This, their, cereals, I've, ordered, fresh,, ...
4        [Whoever, Photoshopped, cookie, front, package...
                               ...                        
49995    [With, many, other, glowing, reviews,, feel, l...
49996    [Giving, stars, because, 'All, Purpose', bake,...
49997    [more, affordable, than, other, bully, sticks,...
49998    [this, daughter, because, love, blood, chocola...
49999    [enough, flavor,, murky, brown, color,, lackin...
Name: Review_1, Length: 50000, dtype: object

In [5]:
from nltk.stem.porter import *
stm = PorterStemmer() #removing endings from words
Token_Data=Token_Data.apply(lambda x:[stm.stem(i) for i in x])
Token_Data

0        [better, than, wolff', kasha., grew, eat, kash...
1        [such, good, product., came, differ, box, desc...
2        [mmmm, chocol, good.<br, />but, some, chocol, ...
3        [thi, their, cereal, i'v, order, fresh,, high,...
4        [whoever, photoshop, cooki, front, packag, tru...
                               ...                        
49995    [with, mani, other, glow, reviews,, feel, litt...
49996    [give, star, becaus, 'all, purpose', bake, mix...
49997    [more, afford, than, other, bulli, sticks,, of...
49998    [thi, daughter, becaus, love, blood, chocol, t...
49999    [enough, flavor,, murki, brown, color,, lack, ...
Name: Review_1, Length: 50000, dtype: object

In [6]:
for i in range(len(Token_Data)):
  Token_Data[i]=' '.join(Token_Data[i]) #conversion into string 
Data['Review_1']=Token_Data

In [7]:
all_words = ' '.join([text for text in Data['Review_1']])
positive_reviews = ' '.join([text for text in Data['Review_1'][Data['Label']==1]])

In [8]:
Negative_reviews=' '.join([text for text in Data['Review_1'][Data['Label']==0]])

#**References:**

  **For Data pre-processing, Logistic Regression and SVM**

  https://www.youtube.com/watch?v=Xh6wFH3Fh7Q&ab_channel=PalestineKrystalHill

# **BOW for all models**

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
BOW_vector = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english') #creating bag of word 
BOW=BOW_vector.fit_transform(Data['Review_1']) 

In [10]:
tfidf_vector =TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english') #creating new feature TFIDF 
tfidf= tfidf_vector.fit_transform(Data['Review_1'])

# **Logistic regression with BOW**

In [11]:
from sklearn.model_selection import train_test_split
train_BOW=BOW[:35000,:]
test_BOW=BOW[35000:,:]
X_train_BOW, X_valid, Y_train_BOW, Y_valid= train_test_split(train_BOW, Train['Label'],random_state=42, test_size=0.3) 
#spliting into the training and testing set

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
regressor=LogisticRegression() 
regressor.fit(X_train_BOW,Y_train_BOW) #training model for the prediction

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [13]:
predict=regressor.predict_proba(X_valid) #making prediction

In [14]:
predict_int=predict[:,1]>=0.3
predict_int=predict_int.astype(np.int)
f1_score(Y_valid, predict_int) #checking the accuracy of the model

0.8777223506509307

# **Logistic regression using TI-IDF**

In [15]:
train_tfidf=tfidf[:35000,:]
test_tfidf=tfidf[35000:,:]
X_train_tfidf, X_valid, Y_train_tfidf, Y_valid= train_test_split(train_tfidf, Train['Label'],random_state=42, test_size=0.3)

In [16]:
regressor.fit(X_train_tfidf, Y_train_tfidf) #logisting regression with TFIDF 
Prediction_1=regressor.predict_proba(X_valid)
Predict_int_1=Prediction_1[:,1]>=0.3
Predict_int_1=Predict_int_1.astype(np.int)

In [17]:
f1_score(Y_valid, Predict_int_1)

0.875364388125409

# **SVM Model using BOW**

In [18]:
from sklearn import svm
SVM_classifier = svm.SVC()
SVM_classifier.fit(X_train_BOW,Y_train_BOW)

SVC()

In [19]:
SVM_prediction=SVM_classifier.predict(X_valid)

In [20]:
SVM_prediction_1=SVM_prediction[:]>=0.3
SVM_prediction_1=SVM_prediction_1.astype(np.int)
f1_score(Y_valid, SVM_prediction_1)

0.8488424875170223

# **SVM model using TF-IDF**

In [21]:
from sklearn import svm
SVM_clf = svm.SVC()
SVM_clf.fit(X_train_tfidf,Y_train_tfidf)

SVC()

In [22]:
SVM_prediction_tfidf=SVM_clf.predict(X_valid)

In [23]:
SVM_prediction_tfidf=SVM_prediction_tfidf[:]>=0.3
SVM_prediction_tfidf=SVM_prediction_tfidf.astype(np.int)
f1_score(Y_valid, SVM_prediction_tfidf)

0.8961783439490446

# **LSTM Network model using Embedding Layer**

In [24]:
# fix random seed for reproducibility
np.random.seed(7)

In [25]:
X=Data.drop('Label',axis=1)
X

,Review,Review_1
0,Better than Wolff's Kasha. I grew up eating Ka...,better than wolff' kasha. grew eat kasha easi ...
1,It was such good product. Came in two differen...,such good product. came differ box describ cov...
2,MMMM Yes all chocolate is good.<br />But some ...,mmmm chocol good.<br />but some chocol better ...
3,"This is, as all of their cereals I've ordered ...","thi their cereal i'v order fresh, high quality..."
4,Whoever Photoshopped the cookie on the front o...,whoever photoshop cooki front packag true tale...
...,...,...
49995,"With so many other glowing reviews, I feel a l...","with mani other glow reviews, feel littl asham..."
49996,Giving it 3 stars because for an 'All Purpose'...,"give star becaus 'all purpose' bake mix, reall..."
49997,"more affordable than other bully sticks, but m...","more afford than other bulli sticks, offic fil..."
49998,I got this for my daughter because she love bl...,thi daughter becaus love blood chocol thi stuf...


In [26]:
Y=Data['Label']
Y

0        1
1        1
2        1
3        1
4        1
        ..
49995    0
49996    0
49997    0
49998    0
49999    0
Name: Label, Length: 50000, dtype: int64

In [27]:
X.shape

(50000, 2)

In [28]:
Y.shape

(50000,)

In [37]:
### Vocabulary size
voc_size=5000

#**References:**
  **for LSTM**

  https://www.youtube.com/watch?v=MXPh_lMRwAI&t=183s&ab_channel=KrishNaik

# **OneHot** **Representation**

In [30]:
mess=X.copy()

In [31]:
mess['Review'][1]

'It was such good product. Came in two different boxes as it is described and covered with plastic wrap together.'

In [32]:
mess.reset_index(inplace=True)

In [33]:
import nltk
import re
from nltk.corpus import stopwords

In [34]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [35]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(mess)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', mess['Review'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
45000
45001
45002
45003
45004
45005
45006
45007
45008
45009
45010
45011
45012
45013
45014
45015
45016
45017
45018
45019
45020
45021
45022
45023
45024
45025
45026
45027
45028
45029
45030
45031
45032
45033
45034
45035
45036
45037
45038
45039
45040
45041
45042
45043
45044
45045
45046
45047
45048
45049
45050
45051
45052
45053
45054
45055
45056
45057
45058
45059
45060
45061
45062
45063
45064
45065
45066
45067
45068
45069
45070
45071
45072
45073
45074
45075
45076
45077
45078
45079
45080
45081
45082
45083
45084
45085
45086
45087
45088
45089
45090
45091
45092
45093
45094
45095
45096
45097
45098
45099
45100
45101
45102
45103
45104
45105
45106
45107
45108
45109
45110
45111
45112
45113
45114
45115
45116
45117
45118
45119
45120
45121
45122
45123
45124
45125
45126
45127
45128
45129
45130
45131
45132
45133
45134
45135
45136
45137
45138
45139
45140
45141
45142
45143
45144
45145
45146
45147
45148
45149
45150
45151
45152
45153
45154
45155
45156
45157
4

In [38]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[752,
  562,
  1523,
  1317,
  839,
  1523,
  1917,
  655,
  2770,
  1535,
  2353,
  1907,
  2525,
  878,
  3041,
  4658,
  3852,
  3939,
  1606,
  4089,
  70,
  4881,
  3117,
  878,
  4553,
  3041,
  4270,
  4471,
  3,
  2704],
 [786, 543, 2503, 2281, 280, 3438, 1933, 3261, 1986, 4295, 4762],
 [3823,
  3749,
  3674,
  786,
  3151,
  3674,
  752,
  1202,
  752,
  1202,
  3674,
  3151,
  3488,
  1917,
  1087,
  89,
  1103,
  1140,
  635,
  3151,
  348,
  130,
  4324],
 [2668,
  888,
  611,
  3956,
  1748,
  3419,
  4132,
  89,
  2668,
  411,
  4823,
  811,
  4888,
  4801,
  4258,
  3949,
  1229,
  639,
  4541,
  839,
  3707,
  3949,
  157,
  3921,
  2499,
  2275,
  986,
  4974,
  4735,
  1886,
  2440,
  2387,
  3852,
  3867,
  2666,
  3910,
  2740,
  839,
  1199,
  4272,
  3494,
  1554,
  4108,
  249,
  3005,
  986,
  2191,
  4084,
  1215,
  3749,
  786],
 [3618,
  4204,
  1812,
  1905,
  4801,
  1824,
  2868,
  315,
  3571,
  122,
  92,
  4287,
  4071,
  4328,
  1812,
  130,
  2079,
 

# **Embedding Representation**

In [39]:
sent_length=20
embedded_matrix=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_matrix)

[[2353 1907 2525 ... 4471    3 2704]
 [   0    0    0 ... 1986 4295 4762]
 [ 786 3151 3674 ...  348  130 4324]
 ...
 [2965 2218 1998 ... 1473  495 1918]
 [   0    0    0 ...   89   89  767]
 [4499 2556  191 ... 2480 4257  157]]


In [40]:
embedded_matrix[0]

array([2353, 1907, 2525,  878, 3041, 4658, 3852, 3939, 1606, 4089,   70,
       4881, 3117,  878, 4553, 3041, 4270, 4471,    3, 2704], dtype=int32)

In [41]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [45]:
len(embedded_matrix),Y.shape


(50000, (50000,))

In [46]:
import numpy as np
X_final=np.array(embedded_matrix)
y_final=np.array(Y)

In [47]:
X_final.shape,y_final.shape


((50000, 20), (50000,))

**Splitting dataser into training and testing set**

In [48]:
X_train_LSTM, X_test_LSTM, y_train_LSTM, y_test_LSTM = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

# Model Traing

In [49]:
model.fit(X_train_LSTM,y_train_LSTM,validation_data=(X_test_LSTM,y_test_LSTM),epochs=10,batch_size=64)

Epoch 1/10
524/524 [==============================] - 18s 20ms/step - loss: 0.4982 - accuracy: 0.7544 - val_loss: 0.4543 - val_accuracy: 0.7900
Epoch 2/10
524/524 [==============================] - 6s 12ms/step - loss: 0.4175 - accuracy: 0.8093 - val_loss: 0.4536 - val_accuracy: 0.7885
Epoch 3/10
524/524 [==============================] - 7s 13ms/step - loss: 0.3848 - accuracy: 0.8273 - val_loss: 0.4607 - val_accuracy: 0.7915
Epoch 4/10
524/524 [==============================] - 6s 12ms/step - loss: 0.3544 - accuracy: 0.8431 - val_loss: 0.4762 - val_accuracy: 0.7832
Epoch 5/10
524/524 [==============================] - 6s 12ms/step - loss: 0.3226 - accuracy: 0.8593 - val_loss: 0.5095 - val_accuracy: 0.7825
Epoch 6/10
524/524 [==============================] - 6s 12ms/step - loss: 0.2894 - accuracy: 0.8756 - val_loss: 0.5411 - val_accuracy: 0.7746
Epoch 7/10
524/524 [==============================] - 7s 12ms/step - loss: 0.2582 - accuracy: 0.8907 - val_loss: 0.5817 - val_accuracy: 0.774

**Dropping out**

In [54]:
from tensorflow.keras.layers import Dropout
import tensorflow as tf
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.compile(loss='mean_squared_error', optimizer='adam', metrics=[tf.keras.metrics.Accuracy()])

# Calculation of accuracy and performance

In [55]:
y_pred_LSTM=model.predict(X_test_LSTM) 
classes_x=np.argmax(y_pred_LSTM,axis=1)

In [56]:
from sklearn.metrics import confusion_matrix

In [57]:
confusion_matrix(y_test_LSTM, classes_x)

array([[8219,    0],
       [8281,    0]])

In [58]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test_LSTM,classes_x)

0.49812121212121213